In [1]:
%matplotlib inline

import itertools
import numpy as np
import os
import random
import sys
import psutil
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
plt.switch_backend('agg')

if "./lib" not in sys.path:
    sys.path.append("./lib")
    
import plotting
from collections import deque, namedtuple
from readingFileEfficiently import *
import VOC2012_npz_files_writter
from DNN import *
from Agent import ObjLocaliser

/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
#This cell reads VOC 2012 dataset and save them in .npz files for future.
#The process of reading data and put them in prper format is time consuming so they are stored in a file.

xml_path = "../VOC2012/Annotations/*.xml"
destination = "../data/"

#It splits dataset to 80% for training and 20% validation.
if not (os.path.isfile(destination+"test_input.npz") or os.path.isfile(destination+"test_target.npz")):
    VOC2012_npz_files_writter.writting_files(xml_path, destination, percentage=0)
    print("Files are ready!!!")
else:
    print("Records are already prepared!!!")

Records are already prepared!!!


In [3]:
num_episodes=10  #200     
replay_memory_size=500000    #2500   
replay_memory_init_size=500 #500  
update_target_estimator_every=10000 #100  
discount_factor=0.99
epsilon_start=1.0
epsilon_end=0.1
epsilon_decay_steps=500000 #10000  
batch_size=32
category = "cat"

#model_name = "defaul_DQL_architecture_epis{}_memorySize{}_UTE{}_EDS{}".format(num_episodes, replay_memory_size, update_target_estimator_every, epsilon_decay_steps)
model_name = "cats"


tf.reset_default_graph()

# Where we save our checkpoints and graphs
experiment_dir = os.path.abspath("./experiments/{}".format(model_name))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
    
# Create estimators
q_estimator = Estimator(scope="q_estimator", summaries_dir=experiment_dir)
target_estimator = Estimator(scope="target_q")

# State processor
state_processor = StateProcessor()



done = False
elist = []
rlist = []


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    # Old API: sess.run(tf.initialize_all_variables())  
    Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

    # The replay memory
    replay_memory = []
    
    num_located = 0

    # Make model copier object
    estimator_copy = ModelParametersCopier(q_estimator, target_estimator)

    # Keeps track of useful statistics
    #stats = plotting.EpisodeStats(
        #episode_lengths=np.zeros(num_episodes),
        #episode_rewards=np.zeros(num_episodes))

    # For 'system/' summaries, usefull to check if currrent process looks healthy
    current_process = psutil.Process()

    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    report_path = os.path.join(experiment_dir, "report")

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(report_path):
        os.makedirs(report_path)
    f = open(report_path+"/log.txt", 'w')
    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)

    # Get the current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())
    #print "init:{}".format(total_t)

    # The epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)

    # The policy we're following
    policy = make_epsilon_greedy_policy(
        q_estimator,
        len(VALID_ACTIONS))
    episode_counter = 0
    #mybreak = 0
    
    for indx,tmp in enumerate(extractData(category, "train", batch_size)):

        #if mybreak > 5:
            #break
        
        img=tmp[0]
        target=tmp[1]

        im2 = Image.frombytes("RGB",(img['image_width'],img['image_height']),img['image'])
        env = ObjLocaliser(np.array(im2),target)
        print "New image is being loaded: {}".format(img['image_filename'])
        
        if len(replay_memory) < replay_memory_init_size:
            
            # Populate the replay memory with initial experience
            print("Populating replay memory...\n")

            env.Reset(np.array(im2))
            state = env.wrapping()


            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)

            for i in range(replay_memory_init_size):

                #env.Reset(np.array(im2))
                #state = env.wrapping()
                #state = state_processor.process(sess, state)
                #state = np.stack([state] * 4, axis=2)
                #action = 0
                #counter = 0
                #done = False

                #while (action != 10) or (counter < 50):

                action_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
                action = np.random.choice(np.arange(len(action_probs)), p=action_probs)

                #next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
                reward = env.takingActions(VALID_ACTIONS[action])
                next_state = env.wrapping()

                if action == 10:
                    done = True
                else: 
                    done = False

                next_state = state_processor.process(sess, next_state)
                next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
                replay_memory.append(Transition(state, action, reward, next_state, done))
                state = next_state

                #counter += 1

                if done:
                    #state = env.reset()
                    env.Reset(np.array(im2))
                    state = env.wrapping()
                    state = state_processor.process(sess, state)
                    state = np.stack([state] * 4, axis=2)
                else:
                    state = next_state 
            
            
            
    
        for i_episode in range(num_episodes):

            # Save the current checkpoint
            saver.save(tf.get_default_session(), checkpoint_path)

            # Reset the environment
            #state = env.reset()
            env.Reset(np.array(im2))
            state = env.wrapping()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)
            loss = None
            t=0
            action = 0
            e = 0
            r = 0
            #done = False
            # One step in the environment
            while (action != 10) and (t < 50):
                #print "hello22:{}".format(loss)
                #env.drawActions()
                # Epsilon for this time step
                epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]

                # Maybe update the target estimator
                if total_t % update_target_estimator_every == 0:
                    estimator_copy.make(sess)
                    print("\nCopied model parameters to target network.")

                # Print out which step we're on, useful for debugging.
                #sys.stdout.flush()

                # Take a step
                #print epsilon
                action_probs = policy(sess, state, epsilon)
                print action_probs
                action = np.random.choice(np.arange(len(action_probs)), p=action_probs)


                #next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
                reward = env.takingActions(VALID_ACTIONS[action])
                next_state = env.wrapping()
                if action == 10:
                    done = True
                else: 
                    done = False


                next_state = state_processor.process(sess, next_state)
                next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)

                # If our replay memory is full, pop the first element
                if len(replay_memory) == replay_memory_size:
                    replay_memory.pop(0)

                # Save transition to replay memory
                replay_memory.append(Transition(state, action, reward, next_state, done))   
                # Update statistics
                #stats.episode_rewards[i_episode] += reward
                #stats.episode_lengths[i_episode] = t

                # Sample a minibatch from the replay memory
                samples = random.sample(replay_memory, batch_size)
                states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*samples))

                # Calculate q values and targets
                q_values_next = target_estimator.predict(sess, next_states_batch)
                targets_batch = reward_batch + np.invert(done_batch).astype(np.float32) * discount_factor * np.amax(q_values_next, axis=1)

                # Perform gradient descent update
                states_batch = np.array(states_batch)
                loss = q_estimator.update(sess, states_batch, action_batch, targets_batch)
                print("Step {} ({}) @ Episode {}/{}, action {}, reward {},loss: {}".format(t, total_t, i_episode + 1, num_episodes, action, reward, loss))
                f.write("Step {} ({}) @ Episode {}/{}, action {}, reward {},loss: {}\n".format(t, total_t, i_episode + 1, num_episodes, action, reward, loss))     


                # Counting number of correct localsied objects
                if reward == 3:
                    num_located += 1

                state = next_state
                t += 1
                total_t += 1
                e = e + loss
                r = r + reward
        
            episode_counter += 1

            # Add summaries to tensorboard
            episode_summary = tf.Summary()
            episode_summary.value.add(simple_value=epsilon, tag="episode/epsilon")
            episode_summary.value.add(simple_value=r, tag="episode/reward")
            episode_summary.value.add(simple_value=t, tag="episode/length")
            episode_summary.value.add(simple_value=current_process.cpu_percent(), tag="system/cpu_usage_percent")
            episode_summary.value.add(simple_value=current_process.memory_percent(), tag="system/v_memeory_usage_percent")
            q_estimator.summary_writer.add_summary(episode_summary, episode_counter)
            q_estimator.summary_writer.flush()

            #prin_stats = plotting.EpisodeStats(
                #episode_lengths=stats.episode_lengths[:i_episode+1],
                #episode_rewards=stats.episode_rewards[:i_episode+1])

            #print("Episode Reward: {} Episode Length: {}".format(prin_stats.episode_rewards[-1], prin_stats.episode_lengths[-1]))
            #f.write("Episode Reward: {} Episode Length: {}".format(prin_stats.episode_rewards[-1], prin_stats.episode_lengths[-1]))
            print("Episode Reward: {} Episode Length: {}".format(r, t))
            f.write("Episode Reward: {} Episode Length: {}".format(r, t))

            elist.append(float(e)/t)
            rlist.append(float(r)/t)
        #mybreak += 1

        
f.close()
print "number of correct located objects:{}".format(num_located)

Instructions for updating:
Please switch to tf.train.get_global_step


/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


input file 1 is loading...
New image is being loaded: 2010_002425.jpg
Populating replay memory...


Copied model parameters to target network.
[0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
Step 0 (0) @ Episode 1/10, action 10, reward 3,loss: 1.49924337864
Episode Reward: 3 Episode Length: 1
[0.09090893 0.09090893 0.09090893 0.09090893 0.09090893 0.09090893
 0.09090893 0.09090893 0.09090893 0.09091073 0.09090893]
Step 0 (1) @ Episode 2/10, action 6, reward 1,loss: 1.52652263641
[0.09090876 0.09090876 0.09090876 0.09090876 0.09090876 0.09090876
 0.09090876 0.09090876 0.09090876 0.09091236 0.09090876]
Step 1 (2) @ Episode 2/10, action 2, reward -1,loss: 1.52098870277
[0.0909086 0.0909086 0.0909086 0.0909086 0.0909086 0.0909086 0.0909086
 0.0909086 0.0909086 0.090914  0.0909086]
Step 2 (3) @ Episode 2/10, action 6, reward 1,loss: 3.0085272789
[0.09090844 0.09090844 0.09090844 0.09090844 0.09090844 0.09090844
 0.0

Step 5 (42) @ Episode 6/10, action 9, reward -1,loss: 1.52020764351
[0.09090205 0.09090205 0.09090205 0.09090205 0.09090205 0.09097945
 0.09090205 0.09090205 0.09090205 0.09090205 0.09090205]
Step 6 (43) @ Episode 6/10, action 5, reward -1,loss: 1.95092189312
[0.09090189 0.09090189 0.09098109 0.09090189 0.09090189 0.09090189
 0.09090189 0.09090189 0.09090189 0.09090189 0.09090189]
Step 7 (44) @ Episode 6/10, action 9, reward -1,loss: 0.924188256264
[0.09090173 0.09090173 0.09090173 0.09090173 0.09090173 0.09090173
 0.09090173 0.09090173 0.09090173 0.09098273 0.09090173]
Step 8 (45) @ Episode 6/10, action 10, reward -3,loss: 1.53539085388
Episode Reward: -7 Episode Length: 9
[0.09090156 0.09090156 0.09090156 0.09098436 0.09090156 0.09090156
 0.09090156 0.09090156 0.09090156 0.09090156 0.09090156]
Step 0 (46) @ Episode 7/10, action 1, reward 1,loss: 1.45343732834
[0.0909014 0.0909014 0.0909014 0.090986  0.0909014 0.0909014 0.0909014
 0.0909014 0.0909014 0.0909014 0.0909014]
Step 1 (47) @

[0.09089502 0.09089502 0.09089502 0.09104982 0.09089502 0.09089502
 0.09089502 0.09089502 0.09089502 0.09089502 0.09089502]
Step 0 (86) @ Episode 10/10, action 6, reward 1,loss: 1.19502651691
[0.09089485 0.09089485 0.09089485 0.09105145 0.09089485 0.09089485
 0.09089485 0.09089485 0.09089485 0.09089485 0.09089485]
Step 1 (87) @ Episode 10/10, action 10, reward 3,loss: 1.71290516853
Episode Reward: 4 Episode Length: 2
New image is being loaded: 2008_005337.jpg
[0.09089469 0.09089469 0.09089469 0.09089469 0.09089469 0.09089469
 0.09089469 0.09105309 0.09089469 0.09089469 0.09089469]
Step 0 (88) @ Episode 1/10, action 4, reward 1,loss: 2.52299022675
[0.09089453 0.09089453 0.09089453 0.09089453 0.09089453 0.09089453
 0.09089453 0.09105473 0.09089453 0.09089453 0.09089453]
Step 1 (89) @ Episode 1/10, action 7, reward -1,loss: 2.20088791847
[0.09089436 0.09089436 0.09089436 0.09089436 0.09089436 0.09089436
 0.09089436 0.09105636 0.09089436 0.09089436 0.09089436]
Step 2 (90) @ Episode 1/10, a

...

Step 27 (383) @ Episode 7/10, action 5, reward -1,loss: 0.977403759956
[0.09084625 0.09084625 0.09084625 0.09084625 0.09084625 0.09084625
 0.09084625 0.09153746 0.09084625 0.09084625 0.09084625]
Step 28 (384) @ Episode 7/10, action 1, reward 1,loss: 2.07148432732
[0.09084609 0.09153909 0.09084609 0.09084609 0.09084609 0.09084609
 0.09084609 0.09084609 0.09084609 0.09084609 0.09084609]
Step 29 (385) @ Episode 7/10, action 6, reward -1,loss: 1.05658447742
[0.09084593 0.09084593 0.09084593 0.09084593 0.09084593 0.09084593
 0.09084593 0.09154073 0.09084593 0.09084593 0.09084593]
Step 30 (386) @ Episode 7/10, action 6, reward -1,loss: 1.762586236
[0.09084576 0.09084576 0.09084576 0.09084576 0.09084576 0.09084576
 0.09084576 0.09154236 0.09084576 0.09084576 0.09084576]
Step 31 (387) @ Episode 7/10, action 8, reward -1,loss: 1.74295330048
[0.0908456 0.0908456 0.0908456 0.0908456 0.0908456 0.0908456 0.0908456
 0.091544  0.0908456 0.0908456 0.0908456]
Step 32 (388) @ Episode 7/10, action 4, rew

Step 0 (425) @ Episode 2/10, action 4, reward 1,loss: 1.29619312286
[0.09083938 0.09083938 0.09083938 0.09083938 0.09083938 0.09083938
 0.09083938 0.09160618 0.09083938 0.09083938 0.09083938]
Step 1 (426) @ Episode 2/10, action 0, reward 1,loss: 1.72959077358
[0.09083922 0.09083922 0.09083922 0.09083922 0.09083922 0.09083922
 0.09083922 0.09160782 0.09083922 0.09083922 0.09083922]
Step 2 (427) @ Episode 2/10, action 7, reward -1,loss: 1.62579202652
[0.09083905 0.09083905 0.09083905 0.09083905 0.09083905 0.09083905
 0.09083905 0.09160946 0.09083905 0.09083905 0.09083905]
Step 3 (428) @ Episode 2/10, action 6, reward -1,loss: 1.54344832897
[0.09083889 0.09083889 0.09083889 0.09083889 0.09083889 0.09083889
 0.09083889 0.09161109 0.09083889 0.09083889 0.09083889]
Step 4 (429) @ Episode 2/10, action 7, reward 1,loss: 1.84018516541
[0.09083873 0.09083873 0.09083873 0.09083873 0.09083873 0.09083873
 0.09083873 0.09161273 0.09083873 0.09083873 0.09083873]
Step 5 (430) @ Episode 2/10, action 4,

Step 11 (468) @ Episode 5/10, action 6, reward -1,loss: 2.08698034286
[0.09083235 0.09167655 0.09083235 0.09083235 0.09083235 0.09083235
 0.09083235 0.09083235 0.09083235 0.09083235 0.09083235]
Step 12 (469) @ Episode 5/10, action 8, reward -1,loss: 1.26870620251
[0.09083218 0.09167818 0.09083218 0.09083218 0.09083218 0.09083218
 0.09083218 0.09083218 0.09083218 0.09083218 0.09083218]
Step 13 (470) @ Episode 5/10, action 1, reward -1,loss: 1.48154163361
[0.09083202 0.09167982 0.09083202 0.09083202 0.09083202 0.09083202
 0.09083202 0.09083202 0.09083202 0.09083202 0.09083202]
Step 14 (471) @ Episode 5/10, action 6, reward -1,loss: 1.47921597958
[0.09083185 0.09168146 0.09083185 0.09083185 0.09083185 0.09083185
 0.09083185 0.09083185 0.09083185 0.09083185 0.09083185]
Step 15 (472) @ Episode 5/10, action 10, reward -3,loss: 1.301684618
Episode Reward: -16 Episode Length: 16
[0.09083169 0.09083169 0.09083169 0.09083169 0.09083169 0.09083169
 0.09083169 0.09083169 0.09083169 0.09083169 0.09

Step 34 (511) @ Episode 7/10, action 3, reward 1,loss: 1.54351890087
[0.09082531 0.09174691 0.09082531 0.09082531 0.09082531 0.09082531
 0.09082531 0.09082531 0.09082531 0.09082531 0.09082531]
Step 35 (512) @ Episode 7/10, action 9, reward -1,loss: 1.13557291031
[0.09082515 0.09174855 0.09082515 0.09082515 0.09082515 0.09082515
 0.09082515 0.09082515 0.09082515 0.09082515 0.09082515]
Step 36 (513) @ Episode 7/10, action 6, reward -1,loss: 1.6006102562
[0.09082498 0.09082498 0.09082498 0.09175018 0.09082498 0.09082498
 0.09082498 0.09082498 0.09082498 0.09082498 0.09082498]
Step 37 (514) @ Episode 7/10, action 9, reward -1,loss: 2.1209602356
[0.09082482 0.09082482 0.09082482 0.09175182 0.09082482 0.09082482
 0.09082482 0.09082482 0.09082482 0.09082482 0.09082482]
Step 38 (515) @ Episode 7/10, action 9, reward -1,loss: 1.42466533184
[0.09082465 0.09082465 0.09082465 0.09175346 0.09082465 0.09082465
 0.09082465 0.09082465 0.09082465 0.09082465 0.09082465]
Step 39 (516) @ Episode 7/10, act

KeyboardInterrupt: 

In [ ]:
plt.xlabel("episods")
plt.ylabel("avg reward per epi")
plt.title("num of correct obj localisation:{0}".format(num_located))
plt.plot(rlist)
plt.savefig("./graphs/reward")
plt.close()

In [ ]:
plt.xlabel("episods")
plt.ylabel("error")
plt.plot(elist)
plt.savefig("./graphs/error")

link to code: https://github.com/dennybritz/reinforcement-learning/blob/master/DQN/Deep%20Q%20Learning%20Solution.ipynb

https://www.oreilly.com/ideas/reinforcement-learning-with-tensorflow